In [744]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import spacy
import en_core_web_sm
import nltk

from nltk.corpus import stopwords, wordnet
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.probability import FreqDist

from heapq import nlargest

In [745]:
data_text = "The Orbiter Discovery, OV-103, is considered eligible for listing in the National Register of Historic Places (NRHP) in the context of the U.S. Space Shuttle Program (1969-2011) under Criterion A in the areas of Space Exploration and Transportation and under Criterion C in the area of Engineering. Because it has achieved significance within the past fifty years, Criteria Consideration G applies. Under Criterion A, Discovery is significant as the oldest of the three extant orbiter vehicles constructed for the Space Shuttle Program (SSP), the longest running American space program to date; she was the third of five orbiters built by NASA. Unlike the Mercury, Gemini, and Apollo programs, the SSP’s emphasis was on cost effectiveness and reusability, and eventually the construction of a space station. Including her maiden voyage (launched August 30, 1984), Discovery flew to space thirty-nine times, more than any of the other four orbiters; she was also the first orbiter to fly twenty missions. She had the honor of being chosen as the Return to Flight vehicle after both the Challenger and Columbia accidents. Discovery was the first shuttle to fly with the redesigned SRBs, a result of the Challenger accident, and the first shuttle to fly with the Phase II and Block I SSME. Discovery also carried the Hubble Space Telescope to orbit and performed two of the five servicing missions to the observatory. She flew the first and last dedicated Department of Defense (DoD) missions, as well as the first unclassified defense-related mission. In addition, Discovery was vital to the construction of the International Space Station (ISS); she flew thirteen of the thirty-seven total missions flown to the station by a U.S. Space Shuttle. She was the first orbiter to dock to the ISS, and the first to perform an exchange of a resident crew. Under Criterion C, Discovery is significant as a feat of engineering. According to Wayne Hale, a flight director from Johnson Space Center, the Space Shuttle orbiter represents a “huge technological leap from expendable rockets and capsules to a reusable, winged, hypersonic, cargo-carrying spacecraft”. Although her base structure followed a conventional aircraft design, she used advanced materials that both minimized her weight for cargo-carrying purposes and featured low thermal expansion ratios, which provided a stable base for her Thermal Protection System (TPS) materials. The Space Shuttle orbiter also featured the first reusable TPS; all previous spaceflight vehicles had a single-use, ablative heat shield. Other notable engineering achievements of the orbiter included the first reusable orbital propulsion system, and the first two-fault-tolerant Integrated Avionics System. As Hale stated, the Space Shuttle remains “the largest, fastest, winged hypersonic aircraft in history,” having regularly flown at twenty-five times the speed of sound."

In [746]:
def is_english_word(word):
    synset = wordnet.synsets(word)
    if synset:
        return True
    else:
        return False

In [747]:
nlp = en_core_web_sm.load()

In [748]:
data_text = data_text.lower()

In [749]:
doc = nlp(data_text)
sentences = [sent.text for sent in doc.sents]
words = [token.text for token in doc]
# words = word_tokenize(data_text)
# sentences = sent_tokenize(data_text)

In [750]:
words = [word for word in words if word.isalpha()]
words = [word for word in words if len(word) > 1]
filtered_words = [word for word in words if is_english_word(word)]

In [751]:
stop_words = stopwords.words('english') + ['iss', 'ii', 'dod']
filtered_words = [word for word in words if word not in stop_words]

In [752]:
lemmatizer = WordNetLemmatizer()
lemmatized_words = [lemmatizer.lemmatize(word) for word in filtered_words]

In [753]:
word_frequencies = FreqDist(lemmatized_words)

In [754]:
print("\nLemmatized Words:")
print(lemmatized_words)

print("\nMost_common Words:")
print(word_frequencies.most_common(8))

print("\nSumm Words:")
print(len(lemmatized_words))


Lemmatized Words:
['orbiter', 'discovery', 'considered', 'eligible', 'listing', 'national', 'register', 'historic', 'place', 'nrhp', 'context', 'space', 'shuttle', 'program', 'criterion', 'area', 'space', 'exploration', 'transportation', 'criterion', 'area', 'engineering', 'achieved', 'significance', 'within', 'past', 'fifty', 'year', 'criterion', 'consideration', 'applies', 'criterion', 'discovery', 'significant', 'oldest', 'three', 'extant', 'orbiter', 'vehicle', 'constructed', 'space', 'shuttle', 'program', 'ssp', 'longest', 'running', 'american', 'space', 'program', 'date', 'third', 'five', 'orbiter', 'built', 'nasa', 'unlike', 'mercury', 'gemini', 'apollo', 'program', 'ssp', 'emphasis', 'cost', 'effectiveness', 'reusability', 'eventually', 'construction', 'space', 'station', 'including', 'maiden', 'voyage', 'launched', 'august', 'discovery', 'flew', 'space', 'thirty', 'nine', 'time', 'four', 'orbiter', 'also', 'first', 'orbiter', 'fly', 'twenty', 'mission', 'honor', 'chosen', 're

In [755]:
sentence_scores = {}
for sentence in sentences:
    for word, freq in word_frequencies.items():
        if word in sentence.lower():
            if sentence not in sentence_scores.keys():
                sentence_scores[sentence] = freq
            else:
                sentence_scores[sentence] += freq

for sentence, scores in sentence_scores.items():
    print(f"{scores} : {sentence}")

64 : the orbiter discovery, ov-103, is considered eligible for listing in the national register of historic places (nrhp) in the context of the u.s. space shuttle program (1969-2011) under criterion a in the areas of space exploration and transportation and under criterion c in the area of engineering.
10 : because it has achieved significance within the past fifty years, criteria consideration g applies.
68 : under criterion a, discovery is significant as the oldest of the three extant orbiter vehicles constructed for the space shuttle program (ssp), the longest running american space program to date; she was the third of five orbiters built by nasa.
33 : unlike the mercury, gemini, and apollo programs, the ssp’s emphasis was on cost effectiveness and reusability, and eventually the construction of a space station.
67 : including her maiden voyage (launched august 30, 1984), discovery flew to space thirty-nine times, more than any of the other four orbiters; she was also the first orb

In [756]:
select_length = int(len(sentences) * 0.3)

summary = nlargest(select_length, sentence_scores, key=sentence_scores.get)

summary

['under criterion a, discovery is significant as the oldest of the three extant orbiter vehicles constructed for the space shuttle program (ssp), the longest running american space program to date; she was the third of five orbiters built by nasa.',
 'including her maiden voyage (launched august 30, 1984), discovery flew to space thirty-nine times, more than any of the other four orbiters; she was also the first orbiter to fly twenty missions.',
 'the orbiter discovery, ov-103, is considered eligible for listing in the national register of historic places (nrhp) in the context of the u.s. space shuttle program (1969-2011) under criterion a in the areas of space exploration and transportation and under criterion c in the area of engineering.',
 'the space shuttle orbiter also featured the first reusable tps; all previous spaceflight vehicles had a single-use, ablative heat shield.',
 'according to wayne hale, a flight director from johnson space center, the space shuttle orbiter represe

# Узагальнив вище написане двома функціями:

In [757]:
def filter_words(text: str) -> list:
    words = word_tokenize(text)

    words = [word for word in words if word.isalpha()]
    words = [word for word in words if len(word) > 1]
    filtered_words = [word for word in words if is_english_word(word)]
    stop_words = stopwords.words('english') + ['iss', 'ii', 'dod']
    filtered_words = [word for word in words if word not in stop_words]
    
    lemmatizer = WordNetLemmatizer()
    lemmatized_words = [lemmatizer.lemmatize(word) for word in filtered_words]

    return lemmatized_words


def nltk_summary(text: str, words: list) -> str:
    sentences = sent_tokenize(text)
    
    word_frequencies = FreqDist(words)

    sentence_scores = {}
    for sentence in sentences:
        for word, freq in word_frequencies.items():
            if word in sentence.lower():
                if sentence not in sentence_scores:
                    sentence_scores[sentence] = freq
                else:
                    sentence_scores[sentence] += freq
    select_length = int(len(sentences) * 0.3)
    summary_sentences = nlargest(select_length, sentence_scores, key=sentence_scores.get)
    return ' '.join(summary_sentences)


filtered_words = filter_words(data_text)
summary_nltk = nltk_summary(data_text, filtered_words)
summary_nltk

'under criterion a, discovery is significant as the oldest of the three extant orbiter vehicles constructed for the space shuttle program (ssp), the longest running american space program to date; she was the third of five orbiters built by nasa. the orbiter discovery, ov-103, is considered eligible for listing in the national register of historic places (nrhp) in the context of the u.s. space shuttle program (1969-2011) under criterion a in the areas of space exploration and transportation and under criterion c in the area of engineering. including her maiden voyage (launched august 30, 1984), discovery flew to space thirty-nine times, more than any of the other four orbiters; she was also the first orbiter to fly twenty missions. the space shuttle orbiter also featured the first reusable tps; all previous spaceflight vehicles had a single-use, ablative heat shield. according to wayne hale, a flight director from johnson space center, the space shuttle orbiter represents a “huge techn